In [16]:
%pip install -qU  langchain langchain_community langchain_nvidia_ai_endpoints langchain_milvus ragas

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


## 載入Emebedding

In [1]:
# from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.embeddings import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name="BAAI/bge-large-zh-v1.5")

# from langchain_nvidia_ai_endpoints import NVIDIAEmbeddings
# embeddings = NVIDIAEmbeddings(model="nemollm-embedding")

/tmp/ipykernel_78070/3662513192.py:4: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="BAAI/bge-large-zh-v1.5")
/usr/local/lib/python3.10/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## 創建Milvus的資料庫
若已有則無須創建

In [2]:
from langchain_milvus import Milvus

# from langchain_community.vectorstores import Milvus

# The easiest way is to use Milvus Lite where everything is stored in a local file.
# If you have a Milvus server you can use the server URI such as "http://localhost:19530".
URI = "./milvus_example.db"

vector_store = Milvus(
    embedding_function=embeddings,
    connection_args={"uri": URI},
    index_params={"index_type": "FLAT", "metric_type": "L2"},
    auto_id=True,
)

## 定義文件內容

In [3]:
from langchain_core.documents import Document

document_1 = Document(
    page_content="I had chocalate chip pancakes and scrambled eggs for breakfast this morning.",
    metadata={"source": "tweet"},
)

document_2 = Document(
    page_content="The weather forecast for tomorrow is cloudy and overcast, with a high of 62 degrees.",
    metadata={"source": "news"},
)

document_3 = Document(
    page_content="Building an exciting new project with LangChain - come check it out!",
    metadata={"source": "tweet"},
)

document_4 = Document(
    page_content="Robbers broke into the city bank and stole $1 million in cash.",
    metadata={"source": "news"},
)

document_5 = Document(
    page_content="Wow! That was an amazing movie. I can't wait to see it again.",
    metadata={"source": "tweet"},
)

document_6 = Document(
    page_content="Is the new iPhone worth the price? Read this review to find out.",
    metadata={"source": "website"},
)

document_7 = Document(
    page_content="The top 10 soccer players in the world right now.",
    metadata={"source": "website"},
)

document_8 = Document(
    page_content="LangGraph is the best framework for building stateful, agentic applications!",
    metadata={"source": "tweet"},
)

document_9 = Document(
    page_content="The stock market is down 500 points today due to fears of a recession.",
    metadata={"source": "news"},
)

document_10 = Document(
    page_content="I have a bad feeling I am going to get deleted :(",
    metadata={"source": "tweet"},
)

documents = [
    document_1,
    document_2,
    document_3,
    document_4,
    document_5,
    document_6,
    document_7,
    document_8,
    document_9,
    document_10,
]
# uuids = [str(uuid4()) for _ in range(len(documents))]

vector_store.add_documents(documents=documents)

[455289468186263552,
 455289468186263553,
 455289468186263554,
 455289468186263555,
 455289468186263556,
 455289468186263557,
 455289468186263558,
 455289468186263559,
 455289468186263560,
 455289468186263561]

In [4]:
text = """\
# 探針卡損壞-Probe Card Damage
## 基本資訊
- **建立日期**: 2018/07/15 05:30:20
- **資訊類別**: 系統連結文件
- **文件類別**: CAR
- **作者**:
  - **姓名**: 1050067 林儀柔 IJLIN1
  - **組織**: 6714 京元電子\總經理室\品質保證處\BU1品保品管部\品質保證三課
## 領域與分類
- **功能領域**: Process Improvement
- **流程領域**: 製造流程->異常處理
- **技術領域**: 設備工程(EE)->LCP
## 客戶與產品
- **客戶與產品型號**: 
## 內容
#16於測試中發生少數dut誤宰，4/2 EE執行手動清針後，續測即發現Wafer下半部All dut fail，確認有P/C damage(PA128AE268-001<br />)及針偏異常.
 
## 附件與參考資料
- **相關附件**:
- **其他參考URL:**
  - http://apmis02.kyec.com.tw/Deltaflow_Client/Request.aspx?activeid=1920247"""

In [5]:
vector_store.add_documents(documents=[Document(page_content=text, metadata={"source": "kyec"})])

[455289468247343124]

In [6]:
from langchain.text_splitter import SentenceTransformersTokenTextSplitter

EMBEDDING_MODEL = "BAAI/bge-large-zh-v1.5"
CHUNK_SIZE = 100
CHUNK_OVERLAP = 20


def get_text_splitter(
    embedding_model_name="BAAI/bge-large-zh-v1.5", chunk_size=506, chunk_overlap=200
) -> SentenceTransformersTokenTextSplitter:
    """Return the token text splitter instance from langchain.

    Returns:
        SentenceTransformersTokenTextSplitter: Splitting text to tokens using sentence model tokenizer
    """
    # Chunksize and chunk overlap can up updated using APP_TEXTSPLITTER_CHUNKSIZE and APP_TEXTSPLITTER_CHUNKOVERLAP respectively
    return SentenceTransformersTokenTextSplitter(
        model_name=embedding_model_name,
        tokens_per_chunk=chunk_size,
        chunk_overlap=chunk_overlap,
    )


text_splitter = get_text_splitter(EMBEDDING_MODEL, CHUNK_SIZE, CHUNK_OVERLAP)

In [7]:
new_documents = text_splitter.create_documents([text], metadatas=[{"source": "kyec"}])

In [8]:
vector_store.add_documents(documents=new_documents)

[455289468697182230,
 455289468697182231,
 455289468697182232,
 455289468697182233,
 455289468697182234]

## 查詢Milvus資料庫內有多少個Chunks?

In [9]:
from pymilvus import Collection, connections

# 連接到 Milvus
connections.connect(uri=URI)
# connections.connect("LangChainCollection", host="localhost", port="19530")

collection_name = vector_store.collection_name
collection = Collection(collection_name)

# 查詢資料總數
print(f"Collection '{collection.name}' has {collection.num_entities} entities.")

Collection 'LangChainCollection' has 16 entities.


## 刪除Milvus資料庫的內文件

In [10]:
filename = "kyec"
## By metadata
vector_store.delete(expr=f"source == '{filename}'")

(insert count: 0, delete count: 6, upsert count: 0, timestamp: 0, success count: 0, err count: 0

## 搜尋資料庫中的文件
### 方法一: vector_store.similarity_search()

In [10]:
K = 5

results = vector_store.similarity_search(
    "LangChain provides abstractions to make working with LLMs easy",
    k=K,
    expr='source == "tweet"',
)

for res in results:
    print(f"* {res.page_content} [{res.metadata}]")

* Building an exciting new project with LangChain - come check it out! [{'pk': 455289468186263554, 'source': 'tweet'}]
* LangGraph is the best framework for building stateful, agentic applications! [{'pk': 455289468186263559, 'source': 'tweet'}]
* I had chocalate chip pancakes and scrambled eggs for breakfast this morning. [{'pk': 455289468186263552, 'source': 'tweet'}]
* I have a bad feeling I am going to get deleted :( [{'pk': 455289468186263561, 'source': 'tweet'}]
* Wow! That was an amazing movie. I can't wait to see it again. [{'pk': 455289468186263556, 'source': 'tweet'}]


### 方法二: retriever.invoke()

In [11]:
retriever = vector_store.as_retriever(
    search_kwargs={"k": K}
)

# retriever = vector_store.as_retriever(
#     search_kwargs={"k": K, "expr": 'source == "tweet"'}
# )

retrieved_docs = retriever.invoke(
    "LangChain provides abstractions to make working with LLMs easy"
)

for res in retrieved_docs:
    print(f"* {res.page_content} [{res.metadata}]")

* Building an exciting new project with LangChain - come check it out! [{'pk': 455289468186263554, 'source': 'tweet'}]
* LangGraph is the best framework for building stateful, agentic applications! [{'pk': 455289468186263559, 'source': 'tweet'}]
* 領 域 * * : 製 造 流 程 - > 異 常 處 理 - * * 技 術 領 域 * * : 設 備 工 程 ( ee ) - > lcp # # 客 戶 與 產 品 - * * 客 戶 與 產 品 型 號 * * : # # 內 容 # 16 於 測 試 中 發 生 少 數 dut 誤 宰 ， 4 / 2 ee 執 行 手 動 清 針 後 ， 續 測 即 發 現 wafer 下 半 部 all du [{'pk': 455289468697182232, 'source': 'kyec'}]
* * : 1050067 林 儀 柔 ijlin1 - * * 組 織 * * : 6714 京 元 電 子 \ 總 經 理 室 \ 品 質 保 證 處 \ bu1 品 保 品 管 部 \ 品 質 保 證 三 課 # # 領 域 與 分 類 - * * 功 能 領 域 * * : process improvement - * * 流 程 領 域 * * : 製 造 流 程 - > 異 常 處 理 - * * 技 術 [{'pk': 455289468697182231, 'source': 'kyec'}]
* # 探針卡損壞-Probe Card Damage
## 基本資訊
- **建立日期**: 2018/07/15 05:30:20
- **資訊類別**: 系統連結文件
- **文件類別**: CAR
- **作者**:
  - **姓名**: 1050067 林儀柔 IJLIN1
  - **組織**: 6714 京元電子\總經理室\品質保證處\BU1品保品管部\品質保證三課
## 領域與分類
- **功能領域**: Process Improvement
- **流程

## 建立RAG的流程

In [12]:
from langchain import hub
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

from langchain_nvidia_ai_endpoints import ChatNVIDIA

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)
    

llm = ChatNVIDIA(
    base_url="http://0.0.0.0:8000/v1",
    model="Qwen/Qwen2.5-7B-Instruct",
    temperature=0.1,
    max_tokens=1000,
    top_p=1.0,
)

prompt = hub.pull("rlm/rag-prompt")

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

/usr/local/lib/python3.10/dist-packages/langsmith/client.py:256: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [13]:
rag_chain.invoke("What is Task Decomposition?")

'I don\'t know the specific context in which "Task Decomposition" is being referred to in the provided information. Task Decomposition generally refers to the process of breaking down a complex task into smaller, more manageable sub-tasks.'

In [14]:
question = "請問探針卡是哪個處室管理的?"
rag_chain.invoke(question)

'探針卡是由京元電子總經理室品質保證處BU1品保品管部品質保證三課管理的。'

In [15]:
sample_queries = [
    "Who introduced the theory of relativity?",
    "Who was the first computer programmer?",
    "What did Isaac Newton contribute to science?",
    "Who won two Nobel Prizes for research on radioactivity?",
    "What is the theory of evolution by natural selection?",
]

In [16]:
dataset = []

for query in sample_queries:
    relevant_docs = retriever.invoke(query)
    response = rag_chain.invoke(query)
    dataset.append(
        {
            "user_input": query,
            "retrieved_contexts": [page.page_content for page in relevant_docs],
            "response": response,
        }
    )

In [17]:
from ragas import EvaluationDataset
evaluation_dataset = EvaluationDataset.from_list(dataset)

In [18]:
from ragas import evaluate
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper


evaluator_llm = LangchainLLMWrapper(llm)
evaluator_embeddings = retriever.vectorstore.embeddings
from ragas.metrics import AnswerRelevancy, Faithfulness, ContextUtilization 

result = evaluate(dataset=evaluation_dataset,
                  metrics=[AnswerRelevancy(), ContextUtilization()],
                  llm=evaluator_llm,
                  embeddings=evaluator_embeddings
                 )
result

Evaluating: 100%|██████████| 10/10 [00:05<00:00,  1.74it/s]


{'answer_relevancy': 0.2000, 'context_utilization': 0.0000}